In [ ]:
library(hash)
library(xts)
library(lubridate)
library(forecast)
library(fpp)

In [1]:
# Constants used throughout the code
INPUT_FILE <- "/home/samarth/workspaces/datakind-workspace/cocUptoJuly2016.csv"
DATA_FOLDER <- "/home/samarth/workspaces/datakind-workspace/analytics/time-series/data/topNComplaints"

# Base Vignette
Purpose:
- To provide a quick start code snippet to get the data, loaded into a useable format for forecasting modules
- Establish a baseline forecast

In [2]:
# load the data
df <- read.csv(INPUT_FILE, stringsAsFactors = F)
df$Complaint.Date <- as.Date(df$Complaint.Date, format = "%m/%d/%Y")
df$Resolution.Date <- as.Date(df$Resolution.Date, format="%m/%d/%Y")
df$NumComplaints <- 1
minDate <- min(df$Complaint.Date)
maxDate <- max(df$Complaint.Date)

head(df)

,X.1,X,Complaint.Type,Area,Locality,Street,Region,Zone,Ward,Complaint.Details,Email,Mobile,Department,Complaint.Date,Resolution.Date,Number.of.days.for.resolution,Range,Is.Resolved,NumComplaints
1,1,1,Mosquito menace,Raja Annamalai Puram,Raja Annamalai Puram,Chamiers Iv Lane,Central Region,N09,N122,PLEASE RECTIFY THE PROBLEM SIR.,00e3902431681886db4cf60de2db9188,f0fbecb87d53088508df2164687a7974,NJ-TEYNAMPET,16679,NA,NA,NA,NO,1
2,2,2,Complaints related to property tax,Mylapore,Mylapore,Justice Sundaram Salai,Central Region,N09,N124,"i remitted the tax arrears of rs 4522 on 21-08-2017 via citi bank card. i have been debited my bank and they confirm the transaction is through. but whereas in the web still it shows as arrears, can you please help me",00e3902431681886db4cf60de2db9188,44c25de05e7654fa0f45741a3391a6d9,NJ-TEYNAMPET,16679,NA,NA,NA,NO,1
3,3,3,Stray cattle,Raja Annamalai Puram,Rohini Garden,Rohini Garden Cross Street,South Region,N13,N173,PLEASE CATCH THE STRAY CATTLE PROBLEM SIR SO PLEASE RECTIFY.,00e3902431681886db4cf60de2db9188,550e0ba50963aa8d8074508918c9fba3,NN-ADYAR,16679,NA,NA,NA,NO,1
4,4,4,Dog menace,Kodungaiyur,K.M Nagar,Perumal St,North Region,N04,N037,SO MANY DOGS ARE HERE SO KINDLY TAKE ACTION SIR.,00e3902431681886db4cf60de2db9188,f0aa1d06f3216e1cc2adef8f8bf1c376,ND-TONDIARPET,16679,NA,NA,NA,NO,1
5,5,5,Desilting of Drain,MADURAVOYAL,KRISHNA NAGAR,KRISHNA NAGAR 24TH STREET,South Region,N11,N147,PLEASE RECTIFY THE PROBLEM.,00e3902431681886db4cf60de2db9188,d1d8af0359f1c13650622e2ceda58886,NL-VALASARAVAKKAM,16679,NA,NA,NA,NO,1
6,6,6,Dog menace,Gopalapuram,Gopalapuram,Ganesh Street,Central Region,N09,N111,SO MANY COWS ARE AROUND IN MY AREA SO PLEASE KINDLY CATCH THAT COWS SIR,00e3902431681886db4cf60de2db9188,e0227e34bff8489fe1c048d0ad97f92b,NJ-TEYNAMPET,16679,NA,NA,NA,NO,1


In [3]:
# pick top complaint types, and model only that data
topComplaintTypes <- data.frame(table(df$Complaint.Type))
topComplaintTypes <- topComplaintTypes[order(-topComplaintTypes$Freq),]
topComplaintTypes <- topComplaintTypes[1:10, ]
topComplaintTypes <- as.character(topComplaintTypes$Var1)
print(topComplaintTypes)

 [1] "Non Burning of Street Lights"                  
 [2] "Dog menace "                                   
 [3] "Removal of garbage"                            
 [4] "Mosquito menace "                              
 [5] "Stagnation of water"                           
 [6] "Complaints related to property tax "           
 [7] "Pot hole fill up/Repairs to the damage surface"
 [8] "Request to relay the Road"                     
 [9] "Removal of Debris"                             
[10] "Encroachment on the public property"           


In [4]:
data <- df[df$Complaint.Type %in% topComplaintTypes, ]

In [5]:
print(unique(data$Complaint.Type))

 [1] "Mosquito menace "                              
 [2] "Complaints related to property tax "           
 [3] "Dog menace "                                   
 [4] "Removal of garbage"                            
 [5] "Non Burning of Street Lights"                  
 [6] "Pot hole fill up/Repairs to the damage surface"
 [7] "Removal of Debris"                             
 [8] "Stagnation of water"                           
 [9] "Request to relay the Road"                     
[10] "Encroachment on the public property"           


# Create data files

For ease of modeling, construct data in the following format: `"Month", "Year", "Complaints"` , with missing values filled in. 


In [7]:
# create the 'ideal' data set
# TODO Sahil, isn't there a better way to do this?
minYear <- year(minDate)
maxYear <- year(maxDate)
ideal <- data.frame(Month=character(), Year=integer(), stringsAsFactors=F)
for(year in seq(from=minYear, to=maxYear)) {    
    for(month in month.abb) {
        r <- nrow(ideal)
        month <- as.character(month)
        ideal[nrow(ideal)+1,] <- c(month, year)        
    }
}

ERROR: Error in eval(expr, envir, enclos): could not find function "year"


ERROR: Error in eval(expr, envir, enclos): could not find function "year"


ERROR: Error in seq(from = minYear, to = maxYear): object 'minYear' not found


In [ ]:
head(ideal)

In [ ]:
constructMonthlyData <- function(data, complaintType) {
    # make this a function for re-use
    d <- data[data$Complaint.Type == complaintType, ]
    # create xts object for rolling up the data 
    series <- xts(d$NumComplaints, d$Complaint.Date)
    series <- apply.monthly(series, FUN = sum)
    # create a df for easy access
    monthlyData <- data.frame(Date=index(series), Complaints=coredata(series))
    # create columns for join 
    monthlyData$Month <- month.abb[month(monthlyData$Date)]
    monthlyData$Year <- year(monthlyData$Date)
    joined <- merge(x = ideal, y = monthlyData, by = c("Month", "Year"), sort=F, all= T)        
    # don't need date
    joined$Date <- NULL
    # sort it by year-month, since R doesn't do it otherwise /endrant    
    joined <- joined[order(as.yearmon(paste0(joined$Year, "-", joined$Month), "%Y-%b")), ]
    joined[is.na(joined$Complaints), ]$Complaints <- 0   
    joined
}
# create the files
for(complaintType in topComplaintTypes) {    
    joined <- constructMonthlyData(data, complaintType)
    # one complaint type has a '/' in it, which messes up the paths    
    path <- file.path(DATA_FOLDER, paste0(gsub("/", "", complaintType), ".csv"))
    print(paste0("Saving file", path))
    write.csv(joined, file=path, row.names=F)
}

# Baseline method

The purpose of this exercise is to establish a [baseline](http://machinelearningmastery.com/how-to-get-baseline-results-and-why-they-matter/) to help us compare the 'naive' method with ML models. 

The modelling methods used are described in detail in [here](https://www.otexts.org/fpp/2/3). 

In [ ]:
# trying it with one complaint type
complaintType <- topComplaintTypes[1]
monthly <- constructMonthlyData(data, complaintType)
monthly$Complaints

In [ ]:
# convert it to a ts object
monthly <- ts(monthly$Complaints, start=c(minYear, 1), frequency = 12)

In [ ]:
print(monthly)

In [ ]:
seasonplot(monthly,ylab="Number of complaints", xlab="Year",
  main=paste0("Seasona plot for ", complaintType),
  year.labels=TRUE, year.labels.left=TRUE, col=1:20, pch=19)

In [ ]:
naiveMethodsPlot <- function(monthly, complaintType) {
    h <- 12
    trainStart <- c(2013, 1)
    trainEnd <- c(2015, 1)
    testStart <- trainEnd
    testEnd <- c(2015, 12)
    monthly2 <- window(monthly,start=trainStart, end=trainEnd)
    monthlyAfter <- window(monthly, start=testStart, end=testEnd)    
    monthlyfit1 <- meanf(monthly2, h=h)
    monthlyfit2 <- naive(monthly2, h=h)
    monthlyfit3 <- snaive(monthly2, h=h)
    monthlyfit4 <- rwf(monthly2, h=h, drift=TRUE)
    
    plot(monthlyfit1, plot.conf=FALSE,
      main=paste0("Forecasts for ", complaintType))    
    lines(monthlyAfter, lty=2)
    lines(monthlyfit2$mean,col=2)
    lines(monthlyfit3$mean,col=3)
    lines(monthlyfit4$mean, col=6)
    legend("topleft",col=c(1,4,2,3,6), lty=c(2, 1, 1, 1,1),
      legend=c("Actual Data", "Pred: Mean method",
               "Pred: Naive method","Pred: Seasonal naive method",
              "Pred: Drift Method"))        
    # TODO: Remove this line, since later on, we'll have all the data for 2015 
    monthlyAfter <- window(monthly, start=c(maxYear, 1), end=c(maxYear, 6))    
    print(paste0(complaintType, ": Mean Method"))
    print(accuracy(monthlyfit1, monthlyAfter))
    print(paste0(complaintType, ": Naive Method"))
    print(accuracy(monthlyfit2, monthlyAfter))
    print(paste0(complaintType, ": Seasonal Method"))
    print(accuracy(monthlyfit3, monthlyAfter))
    print(paste0(complaintType, ": Drift Method"))
    print(accuracy(monthlyfit4, monthlyAfter))
}
naiveMethodsPlot(monthly, topComplaintTypes[1])

In [ ]:
# do this for other complaint types as well
for(complaintType in topComplaintTypes[2:length(topComplaintTypes)]) {
    monthly <- constructMonthlyData(data, complaintType)
    monthly <- ts(monthly$Complaints, start=c(minYear, 1), frequency = 12)
    naiveMethodsPlot(monthly, complaintType)    
}

## Boilerplate Code
The code below contains some boilerplate code that loads the data into a usable format


In [ ]:
loadData <- function(dataFolder) {
    files <- list.files(dataFolder)
    data <- list()
    for(file in files) {    
        df <- read.csv(paste0(dataFolder, "/", file), stringsAsFactors=F)    
        minYear <- min(df$Year)
        complaintType <- substr(file,1,(nchar(file))-4)    
        tsObject <- ts(df$Complaints, start=c(minYear, 1), frequency = 12)
        data[[complaintType]] <- tsObject
    }
    data
}
print(loadData(DATA_FOLDER))